<a href="https://colab.research.google.com/github/Aquiles07/Actividades_Curso/blob/main/Version_Final_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cargar Archivo desde un GITHUB**



In [ ]:
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/Aquiles07/Actividades_Curso/main/estudiantes.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   699  100   699    0     0   3262      0 --:--:-- --:--:-- --:--:--  3266


**Programa Principal**

In [5]:
from datetime import date
import csv

class Persona:
    def __init__(self, nombre="Sin nombre", apellido="Sin apellido", fecha_nacimiento=(1900, 1, 1)):
        self.nombre = nombre
        self.apellido = apellido
        self.fecha_nacimiento = date(*fecha_nacimiento)

    def saludos(self):
        print("Saludos, soy", self.nombre, self.apellido)

    def calcula_edad(self):
        today = date.today()
        edad = int((today - self.fecha_nacimiento).days / 365)
        return edad

class Estudiante(Persona):
    def __init__(self, nombre="Sin nombre", apellido="Sin apellido", fecha_nacimiento=(1900, 1, 1),
                 semestre=1):
        super().__init__(nombre, apellido, fecha_nacimiento)
        self.semestre = semestre
        self.materias = []
        self.calificaciones = {}
        self.__meses__ = {1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo", \
                          6: "Junio", 7: "Julio", 8: "Agosto", 9: "Septiembre", \
                          10: "Octubre", 11: "Noviembre", 12: "Diciembre"}
        self.setMatricula()

    def setMatricula(self):
        if self.fecha_nacimiento.year == 1900:
            print("Error: Fecha de nacimiento inválida")
        else:
            self.matricula = str(self.fecha_nacimiento.year) + self.nombre[:2] + self.apellido[:2]
        return

    def getMatricula(self):
        return self.matricula

    def setFechaIngreso(self, fecha: tuple):
        self.fecha_ingreso = date(*fecha)
        return

    def setSemestre(self, semestre=1):
        self.semestre = semestre
        return

    def getFechaIngreso(self):
        anio = self.fecha_ingreso.year
        mes = self.fecha_ingreso.month
        dia = self.fecha_ingreso.day
        print(f"El estudiante {self.nombre} {self.apellido} ingresó el {dia} de {self.__meses__[mes]} de {anio}")
        return

class Registro():
    def __init__(self, *args: Estudiante, **kwargs):
        if kwargs.get("estudiante") is not None:
            self.estudiante = Estudiante(*kwargs.get("estudiante"))
        else:
            self.estudiante = args[0]  # objeto de tipo Estudiante

    def setCalificaciones(self, **kwargs):
        if len(kwargs) == 0:
            print("Error: Ningún registro de calificaciones proporcionado")
        else:
            if kwargs.get('registros') is not None:
                if len(kwargs.get('registros')) == 0:
                    print("Error: Ningún registro de calificaciones proporcionado")
                else:
                    registros = kwargs.get('registros')
                    for r in registros:
                        self.estudiante.calificaciones[r[0]] = r[1]

            elif kwargs.get('materias') is not None and kwargs.get('calificaciones') is not None:
                materias = kwargs.get('materias')
                califs = kwargs.get('calificaciones')
                if len(materias) != len(califs):
                    print("Error: no hay correspondencia entre el número de materias y el número de calificaciones")
                else:
                    for m, c in zip(materias, califs):
                        self.estudiante.calificaciones[m] = c
        return

    def printCalificaciones(self):
        if len(self.estudiante.calificaciones) == 0:
            print("Error: no se han capturado calificaciones")
        else:
            for m in self.estudiante.calificaciones.keys():
                print(f"{m}: {self.estudiante.calificaciones[m]}")
        return

    def calcularPromedio(self):
        if len(self.estudiante.calificaciones) == 0:
            print("Error: no se han capturado calificaciones para calcular el promedio")
        else:
            promedio = sum(self.estudiante.calificaciones.values()) / len(self.estudiante.calificaciones)
            print(f"Promedio de calificaciones de {self.estudiante.nombre} {self.estudiante.apellido}: {promedio:.2f}")
            return promedio

    @classmethod
    def cargarDesdeCSV(cls, archivo_csv):
        registros_calificaciones = {}
        with open(archivo_csv) as archivo:
            registros = csv.reader(archivo)
            header = next(registros)

            for row in registros:
                nombre = row[1]
                apellido = row[2]
                fecha_nacimiento = tuple(map(int, row[3].strip('()').split(',')))
                semestre = int(row[4])

                # Llamada a la clase estudiante
                estudiante = Estudiante(nombre, apellido, fecha_nacimiento, semestre)

                # Llamada a la clase registro
                registro = cls(estudiante)

                # Obtener las calificaciones del archivo
                registro.setCalificaciones(registros=list(zip(header[5:], [float(calificacion) for calificacion in row[5:]])))

                # Clase registro que obtiene el diccionario nuevo creado
                registros_calificaciones[f"{nombre} {apellido}"] = registro

        return registros_calificaciones


calificaciones_alumnos = "estudiantes.csv"
registros_calificaciones = Registro.cargarDesdeCSV(calificaciones_alumnos)


promedio_semestre = {}

for nombre_estudiante, registro in registros_calificaciones.items():
    # Print individual student's grades
    print(f"\nCalificaciones del estudiante: {nombre_estudiante} del Semestre {registro.estudiante.semestre} con matricula: {registro.estudiante.getMatricula()}:")
    registro.printCalificaciones()
    promedio_semestre.setdefault(registro.estudiante.semestre, []).append(registro.calcularPromedio())

# Print semester averages
for semestre, promedios in promedio_semestre.items():
    promedio_total = sum(promedios) / len(promedios)
    print(f"\nPromedio del semestre {semestre}: {promedio_total:.2f}")



Calificaciones del estudiante: Juan Perez del Semestre 2 con matricula: 1999JuPe:
Fisica: 7.5
Matematicas: 9.0
Programacion: 10.0
Electronica: 8.3
Redes: 6.7
Promedio de calificaciones de Juan Perez: 8.30

Calificaciones del estudiante: Ana Aranda del Semestre 2 con matricula: 1998AnAr:
Fisica: 9.0
Matematicas: 10.0
Programacion: 8.5
Electronica: 8.0
Redes: 9.3
Promedio de calificaciones de Ana Aranda: 8.96

Calificaciones del estudiante: Luis Rojas del Semestre 3 con matricula: 2000LuRo:
Fisica: 7.2
Matematicas: 6.0
Programacion: 8.1
Electronica: 7.6
Redes: 6.5
Promedio de calificaciones de Luis Rojas: 7.08

Calificaciones del estudiante: Diana Valenzo del Semestre 3 con matricula: 2001DiVa:
Fisica: 9.5
Matematicas: 9.1
Programacion: 8.2
Electronica: 8.5
Redes: 10.0
Promedio de calificaciones de Diana Valenzo: 9.06

Calificaciones del estudiante: Lorena Diaz del Semestre 2 con matricula: 1999LoDi:
Fisica: 6.3
Matematicas: 7.4
Programacion: 9.0
Electronica: 8.2
Redes: 7.6
Promedio de 